In [1]:
import numpy as np

In [2]:
fmaps = 2
channels = 3
k_y = 2
k_x = 2

H_in = 3
W_in = 3
H_out = H_in - k_y + 1
W_out = W_in - k_x + 1

H_xu = channels*k_y*k_x
W_xu = H_out*W_out

## Set up weights

In [3]:
w1 = [
    [[1,1],[2,2]],
    [[1,1],[1,1]],
    [[0,1],[1,0]]
]
w2 = [
    [[1,0],[0,1]],
    [[2,1],[2,1]],
    [[1,2],[2,0]]
]

In [4]:
w = np.array([w1, w2])
print(w.shape)
assert w.shape == (fmaps, channels, k_y, k_x)

(2, 3, 2, 2)


In [5]:
w_re = w.reshape((fmaps, channels * k_y * k_x))

## Set up X

In [6]:
x_arr = [
    [[1,2,0],[1,1,3],[0,2,2]],
    [[0,2,1],[0,3,2],[1,1,0]],
    [[1,2,1],[0,1,3],[3,3,2]]
]

In [7]:
x = np.array(x_arr)
print(x.shape)
assert x.shape == (channels, H_in, W_in)

(3, 3, 3)


## Set up x_unrolled

In [8]:
x_unr = np.empty([H_xu, W_xu])
print(x_unr.shape)

(12, 4)


In [9]:
for tId in range(channels * H_out * W_out):
    if tId < channels * W_xu:
        channel = tId // W_xu
        unroll_col = tId % W_xu

        # indices in x
        x_row = unroll_col // W_out
        x_col = unroll_col % W_out

        # indices in x_unroll
        h_unroll = x_row * W_out + x_col
        w_base = channel * k_y * k_x
        
#         print('\ntId=%d' % tId)
#         print('channel=%d' % channel)
#         print('unroll_col=%d' % unroll_col)
#         print('x_row=%d' % x_row)
#         print('x_col=%d' % x_col)
#         print('h_unroll=%d' % h_unroll)
#         print('w_base=%d' % w_base)
#         print('---')

        ky = 0
        while ky < k_y:
            kx = 0
            while kx < k_x:
                w_unroll = w_base + ky * k_x + kx
                print('w_unroll=%d' % w_unroll)
                x_unr[w_unroll, h_unroll] = x[channel, x_row + ky, x_col + kx]
                kx += 1
            ky += 1

    else:
        print('tId=%d not valid!' % tId)

w_unroll=0
w_unroll=1
w_unroll=2
w_unroll=3
w_unroll=0
w_unroll=1
w_unroll=2
w_unroll=3
w_unroll=0
w_unroll=1
w_unroll=2
w_unroll=3
w_unroll=0
w_unroll=1
w_unroll=2
w_unroll=3
w_unroll=4
w_unroll=5
w_unroll=6
w_unroll=7
w_unroll=4
w_unroll=5
w_unroll=6
w_unroll=7
w_unroll=4
w_unroll=5
w_unroll=6
w_unroll=7
w_unroll=4
w_unroll=5
w_unroll=6
w_unroll=7
w_unroll=8
w_unroll=9
w_unroll=10
w_unroll=11
w_unroll=8
w_unroll=9
w_unroll=10
w_unroll=11
w_unroll=8
w_unroll=9
w_unroll=10
w_unroll=11
w_unroll=8
w_unroll=9
w_unroll=10
w_unroll=11


In [10]:
print(x_unr)

[[ 1.  2.  1.  1.]
 [ 2.  0.  1.  3.]
 [ 1.  1.  0.  2.]
 [ 1.  3.  2.  2.]
 [ 0.  2.  0.  3.]
 [ 2.  1.  3.  2.]
 [ 0.  3.  1.  1.]
 [ 3.  2.  1.  0.]
 [ 1.  2.  0.  1.]
 [ 2.  1.  1.  3.]
 [ 0.  1.  3.  3.]
 [ 1.  3.  3.  2.]]


In [11]:
print(w_re)

[[1 1 2 2 1 1 1 1 0 1 1 0]
 [1 0 0 1 2 1 2 1 1 2 2 0]]


In [12]:
np.dot(w_re, x_unr)

array([[ 14.,  20.,  15.,  24.],
       [ 12.,  24.,  17.,  26.]])

## Larger unroll

In [13]:
batches = 2

In [14]:
x1 = [
    [[1,2,0],[1,1,3],[0,2,2]],
    [[0,2,1],[0,3,2],[1,1,0]],
    [[1,2,1],[0,1,3],[3,3,2]]
]
x2 = [
    [[1,2,1],[0,1,3],[3,3,2]],
    [[1,2,0],[1,1,3],[0,2,2]],
    [[0,2,1],[0,3,2],[1,1,0]]
]

In [15]:
bigx = np.array([x1,x2])
print(bigx.shape)
assert bigx.shape == (batches, channels, H_in, W_in)
print(bigx)

(2, 3, 3, 3)
[[[[1 2 0]
   [1 1 3]
   [0 2 2]]

  [[0 2 1]
   [0 3 2]
   [1 1 0]]

  [[1 2 1]
   [0 1 3]
   [3 3 2]]]


 [[[1 2 1]
   [0 1 3]
   [3 3 2]]

  [[1 2 0]
   [1 1 3]
   [0 2 2]]

  [[0 2 1]
   [0 3 2]
   [1 1 0]]]]


In [16]:
bigH_xu = channels*k_y*k_x
bigW_xu = batches*H_out*W_out

In [17]:
bigx_unr = np.empty([bigH_xu, bigW_xu])
print(bigx_unr.shape)

(12, 8)


In [18]:
for batch in range(batches):
    for tId in range(channels * H_out * W_out):
        if tId < channels * W_xu:
            channel = tId // W_xu
            unroll_col = tId % W_xu

            # indices in x
            x_row = unroll_col // W_out
            x_col = unroll_col % W_out

            # indices in x_unroll
            h_unroll = x_row * W_out + x_col
            w_base = channel * k_y * k_x

            ky = 0
            while ky < k_y:
                kx = 0
                while kx < k_x:
                    w_unroll = w_base + ky * k_x + kx
                    bigx_unr[w_unroll, (batch * W_xu) + h_unroll] = bigx[batch, channel, x_row + ky, x_col + kx]
                    kx += 1
                ky += 1

        else:
            print('tId=%d not valid!' % tId)

In [19]:
print(bigx_unr)

[[ 1.  2.  1.  1.  1.  2.  0.  1.]
 [ 2.  0.  1.  3.  2.  1.  1.  3.]
 [ 1.  1.  0.  2.  0.  1.  3.  3.]
 [ 1.  3.  2.  2.  1.  3.  3.  2.]
 [ 0.  2.  0.  3.  1.  2.  1.  1.]
 [ 2.  1.  3.  2.  2.  0.  1.  3.]
 [ 0.  3.  1.  1.  1.  1.  0.  2.]
 [ 3.  2.  1.  0.  1.  3.  2.  2.]
 [ 1.  2.  0.  1.  0.  2.  0.  3.]
 [ 2.  1.  1.  3.  2.  1.  3.  2.]
 [ 0.  1.  3.  3.  0.  3.  1.  1.]
 [ 1.  3.  3.  2.  3.  2.  1.  0.]]


In [20]:
yu = np.dot(w_re, bigx_unr)
yu

array([[ 14.,  20.,  15.,  24.,  12.,  21.,  21.,  25.],
       [ 12.,  24.,  17.,  26.,  13.,  24.,  16.,  23.]])

In [21]:
bigy = np.empty([batches * fmaps, W_xu])
print(bigy.shape)

(4, 4)


In [31]:
for batch in range(batches):
    for tId in range(fmaps * H_out * W_out):
        if tId < fmaps * W_xu:
            yu_row = tId // W_xu;
            yu_col = tId // W_xu + (batch * W_xu)

            y_row = yu_row + (batch * fmaps)
            y_col = tId // W_xu
            
            print(f'yu_row={yu_row} yu_col={yu_col} y_row={y_row} y_col={y_col}')

            bigy[y_row, y_col] = yu[yu_row, yu_col]

yu_row=0 yu_col=0 y_row=0 y_col=0
yu_row=0 yu_col=0 y_row=0 y_col=0
yu_row=0 yu_col=0 y_row=0 y_col=0
yu_row=0 yu_col=0 y_row=0 y_col=0
yu_row=1 yu_col=1 y_row=1 y_col=1
yu_row=1 yu_col=1 y_row=1 y_col=1
yu_row=1 yu_col=1 y_row=1 y_col=1
yu_row=1 yu_col=1 y_row=1 y_col=1
yu_row=0 yu_col=4 y_row=2 y_col=0
yu_row=0 yu_col=4 y_row=2 y_col=0
yu_row=0 yu_col=4 y_row=2 y_col=0
yu_row=0 yu_col=4 y_row=2 y_col=0
yu_row=1 yu_col=5 y_row=3 y_col=1
yu_row=1 yu_col=5 y_row=3 y_col=1
yu_row=1 yu_col=5 y_row=3 y_col=1
yu_row=1 yu_col=5 y_row=3 y_col=1


In [32]:
bigy

array([[ 14.,  20.,  15.,  24.],
       [ 12.,  24.,  21.,  25.],
       [ 12.,  24.,  17.,  26.],
       [ 13.,  24.,  16.,  23.]])